<a href="https://colab.research.google.com/github/yeijSong/HandsOnDataAnalysis/blob/main/08_%ED%83%80%EC%8A%88_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 임포트

In [ ]:
from urllib.request import urlopen # 좀 더 간편한 url 불러오는 라이브러리(공공기관) but 사기업에서 운영하는 복잡하고 보안이 강한 사이트는 selenium 사용
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# 크롤링 정보를 저장할 데이터프레임 선언
data = pd.DataFrame(columns=['스테이션명', '위치', '상태정보', '위도', '경도'])

# 크롤링 할 타슈 사이트 주소
url_tashu = 'https://www.tashu.or.kr/userStationAction.do?process=list'

# 페이지 이동 시 추가로 적어줘야 하는 url
url_add = '&page='

# 전체 페이지 수
total_page = 27

# 27페이지에 대한 정보를 추출하는 for문
for index in range(1,total_page+1) :
  if index == 1 :
    url = url_tashu # 첫번째 페이지
  else :
    url = url_tashu + url_add + str(index) # 페이지 정보 추가

  # url에서 HTML 추출
  html = urlopen(url)

  # HTML을 파싱하여 데이터로 변환
  bsObject = BeautifulSoup(html, 'html.parser', from_encoding='utf-8') # 내가 가져온 정보가 html이니까 html.parser를 통해 파싱해줘

  # history_table이라고 하는 class를 가진 table만 추출
  table = bsObject.find_all('table',{'class','history_table'})

  # table내에서 tr 추출
  tr = table[0].find_all('tr')

  # 첫 tr은 contents가 아니라, 데이터 정보를 의미하기 떄문에 제외
  tr = tr[1:len(tr)]

  # 한페이지의 정보를 크롤링
  for index_tr in range(0,len(tr)):
    td = tr[index_tr].find_all('td')

    # 장소 정보 (ex_ 1.무역전시관입구(택시승강장) 형태로 장소정보가 있기 때문에 .으로 split)
    station = td[0].text.split('.')[1]

    # 위치 정보
    location = td[1].text

    # 상태 정보
    condition = td[2].text

    # 좌표 추출
    # <a href="javascript:locationModal('36.374389', '127.392375');" 에서 '36.374389', '127.392375');" 추출
    geo = td[3].a.attrs['href'].split('(')[1] 
    
    # 위도, 경도 정보 추출을 위한 데이터 클렌징
    geo = re.sub('[;)]','',geo) #'36.374389', '127.392375');" -> '36.374389', '127.392375' 추출
    detail_location = geo.replace("'","") # 작은 따옴표('')삭제

    # 위도 Latitude
    lat = detail_location.split(',')[0]

    # 경도 Longitude
    lon = detail_location.split(',')[1]

    # data에 크롤링 결과 저장
    data = data.append({'스테이션명' : station,
                        '위치' : location,
                        '상태정보' : condition,
                        '위도' : lat,
                        '경도' : lon},
                       ignore_index=True)
    
    print('Complete of ' + str(index_tr+1) + 'information with' + str(index) + 'Page')
  
  # 크롤링(Crwaling)막히는 것 방지
  time.sleep(1)

print('-'*30)
print(data)




Complete of 1information with1Page
Complete of 2information with1Page
Complete of 3information with1Page
Complete of 4information with1Page
Complete of 5information with1Page
Complete of 6information with1Page
Complete of 7information with1Page
Complete of 8information with1Page
Complete of 9information with1Page
Complete of 10information with1Page
Complete of 1information with2Page
Complete of 2information with2Page
Complete of 3information with2Page
Complete of 4information with2Page
Complete of 5information with2Page
Complete of 6information with2Page
Complete of 7information with2Page
Complete of 8information with2Page
Complete of 9information with2Page
Complete of 10information with2Page
Complete of 1information with3Page
Complete of 2information with3Page
Complete of 3information with3Page
Complete of 4information with3Page
Complete of 5information with3Page
Complete of 6information with3Page
Complete of 7information with3Page
Complete of 8information with3Page
Complete of 9infor

크롤링 저장하기

In [ ]:
data.to_csv('TASHU.csv', encoding='utf-8-sig')